In [4]:
#1
import requests
from bs4 import BeautifulSoup

html = requests.get('https://comic.naver.com/webtoon')
soup = BeautifulSoup(html.text, 'html.parser')

data = soup.findAll('div' , {'class' : "WeekdayMainView__daily_all_wrap--UvRFc"})
print(len(data))

0


In [1]:
#2
import requests
from bs4 import BeautifulSoup
import time

time.sleep(3)
url = 'https://comic.naver.com/webtoon'
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    data = soup.findAll('div', {'class': "WeekdayMainView__daily_all_wrap--UvRFc"})
    print(len(data))

else:
    print(f"Http 요청이 실패했습니다. 응답코드: {response.status_code}")

#네이버웹툰을 웹에서 접근하기가 어려웠습니다.

0


In [7]:
#3
#참고사이트:'https://velog.io/@beegoat/Web-%EC%9B%B9%EC%84%9C%EB%B2%84-API-%EC%84%9C%EB%B2%84'
#API sever로 접근하기위해 api 주소로 접근을 시도하였습니다.
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import json

browser = webdriver.Chrome()

url = 'https://comic.naver.com/api/webtoon/titlelist/weekday?order=user'.format('user')
sada = browser.get(url)
time.sleep(3)
source = browser.page_source
soup = BeautifulSoup(source, 'html.parser')

soup #여기서 soup객체는 json으로 읽을 수 없었습니다.



<html><head><meta content="light dark" name="color-scheme"/><meta charset="utf-8"/></head><body style="margin: 0"><div></div><pre>{"titleListMap":{"TUESDAY":[{"titleId":814543,"titleName":"너는 그냥 개그만화나 그려라","author":"조석","thumbnailUrl":"https://image-comic.pstatic.net/webtoon/814543/thumbnail/thumbnail_IMAG21_b9d0083f-ced5-4ba8-ac33-c685b96d611a.jpg","up":false,"rest":false,"bm":false,"adult":false,"starScore":9.96106,"viewCount":0,"openToday":false,"potenUp":false,"bestChallengeLevelUp":false,"finish":false,"new":true},{"titleId":783053,"titleName":"김부장","author":"박태준 만화회사 / 정종택","thumbnailUrl":"https://image-comic.pstatic.net/webtoon/783053/thumbnail/thumbnail_IMAG21_d7732f14-26da-4e35-8762-660cc87b53f1.jpg","up":false,"rest":false,"bm":false,"adult":false,"starScore":9.47738,"viewCount":0,"openToday":false,"potenUp":false,"bestChallengeLevelUp":false,"finish":false,"new":false},{"titleId":784248,"titleName":"내가 키운 S급들","author":"seri / 비완 / 근서","thumbnailUrl":"https://image-comic.pst

In [9]:
#4
#참고사이트:'https://dojang.io/mod/page/view.php?id=2469'
import aiohttp #비동기 HTTP클라이언트 라이브러리(웹 요청을 처리하기위해 사용)
import asyncio #비동기 프로그래밍을 지원하는 라이브러리
import nest_asyncio #asyncio를 사용하는 코드를 jupyter notebook과 같은 환경에서 실행할 수 있게 해주는 코드
import json #json 데이터를 파싱하기 위한 모듈(json은 텍스트 포맷)
import pandas as pd #데이터를 다루기 위한 데이터 분석 라이브러리

nest_asyncio.apply() #asyncio를 사용하는 코드를 Jupyer Noterbook과 같은 환경에서 실행할 수 있도록 설정해줌


async def fetch_data():
    url = "https://comic.naver.com/api/webtoon/titlelist/weekday?order=user".format('user')  
  

    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                return await response.text()
            else:
                print(f"Error with status code: {response.status}")
                return None

async def main():
    data = await fetch_data()
    return data

loop = asyncio.get_event_loop() #무한 루프를 돌며 매 루프마다 하나씩 실행시키는 방식입니다.

# 코루틴 실행
data = loop.run_until_complete(main()) #데이터를 비동기적으로 가져오기
parsed_json = json.loads(data)  # data가 이미 문자열이므로 이렇게 사용 가능
#json문자열을 python객체로 변환

days = ['MONDAY', 'TUESDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY']
frames = []

for day in days:
    df_day = pd.DataFrame(parsed_json['titleListMap'][day])
    df_day['dayOfWeek'] = day  # 신규 컬럼 추가(요일은 따로 모 클래스가 존재하지않아서 새로 생성했습니다.)
    frames.append(df_day) #그 이후에 빈리스트에 저장했습니다.

# 모든 데이터 프레임을 하나로 합침
result_df = pd.concat(frames)

# 원하는 열만 남기고 나머지 열 제거
result_df = result_df.drop(['titleId','thumbnailUrl', 'up', 'rest', 'bm', 'adult', 'starScore', 'viewCount', 'openToday', 'potenUp', 'bestChallengeLevelUp', 'finish', 'new'], axis = 'columns')
result_df


,titleName,author,dayOfWeek
0,참교육,채용택 / 한가람,MONDAY
1,뷰티풀 군바리,설이 / 윤성원,MONDAY
2,신의 탑,SIU,MONDAY
3,똑 닮은 딸,이담,MONDAY
4,백수세끼,치즈,MONDAY
...,...,...,...
84,네 것이었던 것,아이아리 / 바밤,SUNDAY
85,데빌샷,CTK,SUNDAY
86,어떤소란,케첩,SUNDAY
87,패션쇼,로커8,SUNDAY
